In [13]:
##imports

import pandas as pd
import numpy as np
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")


# Acquire and Prep

In [14]:
# use data supplied
df = pd.read_table("~/Downloads/anonymized-curriculum-access.txt",
                    sep = '\s', 
                    header = None, 
                    names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])
# drop na
df.dropna(inplace=True)
# turn to datetime
df['date'] = pd.to_datetime(df['date'])
#df = df.set_index("date")
# set cohort to an object
df.cohort = df.cohort.astype(int).astype(object)
#create copy to reference
df0 = df.copy()
df=df0.copy()
#seperate out to determine lesson
df = df[(df.page.str.count("/")>1)]
df["lesson"] = df.page#.str.split("/").str[-1]
#reduce what is returned to be more relevant to lessons
df = df[(df["page"].str.contains("1|2|'|grades|cohort|student|user|.wof|.p|.html|.git|%|www|5c|understand|login|setup|.jpg|toc|.py|/13|.json|user_input|vocab|where|.md|wp-admin")==False)]
# wanted words not symbols
df = df[df.lesson.str.len() > 2]
# drop anything that turned weird
df = df.dropna(subset=["lesson"])
# split and return last section of split
df.lesson = df.lesson.str.split("/").str[-1]
#make copy for reference
df1 = df.copy()

# 1 - Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [91]:
# set to copy, reduce return,split up
df = df0.copy()
df = df[(df["page"].str.contains("login|student|understand|.jpg|toc|.py|/13|.json|user_input|vocab|where|.md|wp-admin")==False)]
df = df[df["page"]!="/"]
# sort and filter down, returning the most visited sites as seen by each cohort
most_by_cohort = df.groupby(["cohort",'page'])['cohort']\
                    .agg(["count"])\
                        .reset_index()\
                            .groupby(["cohort"])["page","count"].max()\
                                .groupby(["page"])\
                                    .count()\
                                        .sort_values(by="count",ascending=False)\
                                            .rename(columns={"count":"Most Visited for (Count) Cohorts"})
most_by_cohort.head(1)

,Most Visited for (Count) Cohorts
page,
web-design/ux/purpose,22


most cohorts visted web-design/ux/purpose the most

In [98]:
# plot out an visits per cohort for the highest visited pages
fig = px.bar( data_frame=df[df.page.isin(most_by_cohort.index)].groupby(["cohort",'page'])['cohort']\
                    .agg(["count"])\
                        .reset_index().sort_values(by="count",ascending=False),
        x="cohort",
        y="count",
        color="page",
        title="Cohort Visit Count",
        barmode="relative")

fig.update_xaxes(type='category')
fig.update_layout(xaxis_title="Cohort ID", yaxis_title="Visit Count",font=dict(size=12))


# 2 - Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [17]:
# reset to copy, and sort down, creating a pivot of the lessons by cohort
df=df1.copy()
df = df.groupby(["cohort","lesson"])[["date"]].count().reset_index().rename(columns={"date":"count"})
df = pd.pivot_table(df,values="count",index=["cohort"],columns=["lesson"],aggfunc=sum,margins=True)
#drop the last created from margin
df.drop(df.tail(1).index,inplace=True)
#loop through and create as a percent to determine relevance for each, 
# dropping columns where cohorts viewed within threshold (.5)
for x in df.drop(columns=["All"]).columns.tolist():
    df[x]=df[x]/df["All"]
    if (df[x].max() - df[x].mean()) < 0.5:
        df.drop(columns=x,inplace=True)
#drop the other margin
df.drop(columns="All",inplace=True)
#return the max finds
df.idxmax()

lesson
attributes-and-css    139
intro                  13
dtype: int64

#### Cohort 139 for lesson "Attributes and CSS" (it's the only one they took)  
#### Cohort 13 for lesson "Intro"

# 3 - Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [25]:
#set to fresh
df=df0.copy()
#find first and last dates, merging into main dataframe
first_access = df.groupby('id')['date'].min().reset_index().rename(columns={"date":"start"})
last_access = df.groupby('id')['date'].max().reset_index().rename(columns={"date":"last"})
df = pd.merge(df,first_access,how="left",on="id")
df = pd.merge(df,last_access,how="left",on="id")
#get the ttl time to sort by
df["ttl_time"] = df["last"]-df["start"]
#only want people who were accessing for more than one day
pages_per_id = df[df["ttl_time"]>"1 days"].groupby("id")[["page"]].count()
#finding low views (bottom 2%) for students who lasted more than one day
low_views = pages_per_id[pages_per_id.page < pages_per_id.page.quantile(.02)]
#reset to index and filter to those students
df = df[df["id"].isin(low_views.index)]
#return some results and make some speculation
df["id"].value_counts(ascending=True)[:5],df["ttl_time"].value_counts()[:5],df["cohort"].value_counts()[:5]


(573    2
 980    3
 169    5
 399    5
 115    6
 Name: id, dtype: int64,
 6 days      12
 9 days      12
 85 days     12
 272 days    12
 104 days    11
 Name: ttl_time, dtype: int64,
 19    28
 18    22
 1     18
 28    14
 15    12
 Name: cohort, dtype: int64)

#### Significant amount are from cohorts 19 and 18... possibly a different style of teaching during this time?

# 4 - Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [19]:
#set to fresh
df = df0.copy()
# find id's who visited a lot of unique pages or from a lot of unique ips
id_df = df.groupby(['id'])["page","ip"].nunique()
#filter down to those users who are outside 3sigma of mean
high_users = id_df[(id_df.page>id_df.page.mean()+(id_df.page.std()*3))|(id_df.ip>id_df.ip.mean()+(id_df.ip.std()*3))]
#return
high_users

,page,ip
id,,
1,976,35
11,871,32
64,436,34
127,296,67
226,168,83
228,145,148
248,625,11
314,225,49
337,173,57


#### Users are accessing much more pages than the average user (signifying possibility of scraping), as well as from many unique IP addresses. Could also be instructors, so let's investigate IP address

In [20]:
#looking at highest value counts of ip
df=df0.copy()
df.ip.value_counts()[:2]

97.105.19.58    268648
97.105.19.61     60530
Name: ip, dtype: int64

#### 97.105.19 is likely associated with codeup due to the number of hits (presumably scanning/back-ups as well as student use)

In [21]:
#set fresh
df=df0.copy()
#filter dataframe to users we've identified and drop ip that is likely Codeup
df = df[(df["id"].isin(high_users.index)) & (df["ip"].str.contains("97.105.19.58|97.105.19.61")==False)]
#grouping down for report
df.groupby(["ip","id"])["page"].count()[df.groupby(["ip","id"])["page"].count()>1000]

ip               id 
172.124.67.93    570    2530
173.173.115.201  248    2599
70.121.215.53    1      1061
70.125.129.152   314    1538
71.150.217.33    64     6790
72.178.123.235   314    1274
76.185.131.226   11     2618
76.185.145.231   11     4754
Name: page, dtype: int64

#### ruling out the 97.106.19, i would investigate futher into the other IP/IDs (other locations, business use, etc)

# 6 - What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [29]:
#refresh
df = df0.copy()
#filter down pages
df = df[(df["page"].str.contains("students|understand|login|setup|.jpg|toc|.py|/13|.json|user_input|vocab|where|.md|wp-admin")==False)]
df = df[df["page"]!="/"]
df = df[df.page.str.len()>2]
#get first access dates
first_access = df.groupby('cohort')["date"].min()
first_access.reset_index()
#merge and find elapsed time
df = pd.merge(df,first_access,how="left",on="cohort")
df.rename(columns={"date_y":"start_date"},inplace=True)
df["elapsed_time"] = df["date_x"] - df["start_date"]
#restrict elapsed time to something reasonable
df = df[df.elapsed_time>"180 days"].groupby(["page","cohort"])[["cohort"]].count().rename(columns={"cohort":"count"}).reset_index()
#return the most visted by filtered cohort
df.sort_values(by="count",ascending=False)["page"].head(5)


5167    javascript-i
4868        java-iii
7331          spring
3884        html-css
4582         java-ii
Name: page, dtype: object

# 7 - Which lessons are least accessed?

In [23]:
#refresh and set
df = df1.copy()
#filter down
df.groupby(["lesson"]).count()[["page"]].sort_values(by="page",ascending=True).head(10).index.tolist()

['function',
 'extra-exercises',
 'cssasxz',
 'if-statement',
 'limit-order-by',
 'cli',
 'ntellij',
 'css',
 'functions',
 '07-editing-files-with-vim']

In [102]:
df = df0.copy()

df = df[["id","time"]].groupby("id").agg(["count","nunique"]).sort_values(by=("time","nunique"),ascending=False).droplevel(axis=1,level=0)
df["nunique"].mean()

876.6388583973655

In [106]:
df=df.head()
fig = px.bar( data_frame = df,
        x=df.index,
        y="nunique",
        title="IDs Visited Most (Mean: < 1k)")
fig.update_xaxes(type='category')
fig.update_layout(xaxis_title="User ID", yaxis_title="Visit Count",font=dict(size=24))
